# Payoff d'un strangle long sur SPY

Ce TP documente le payoff d'un strangle européen (put OTM + call OTM) sur SPY.
Objectifs pédagogiques :
1. Examiner l'historique de SPY sur un an pour fixer un spot de référence.
2. Définir les strikes d'un strangle (put plus bas, call plus haut) autour du spot.
3. Tracer la courbe de payoff en fonction du spot et relier la formule à la visualisation.

In [ ]:
%pip -q install yfinance matplotlib pandas numpy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8-darkgrid")

def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_put(spot: float, strike: float) -> float:
    return max(strike - spot, 0.0)

def payoff_strangle(spot: float, k_put: float, k_call: float) -> float:
    return payoff_put(spot, k_put) + payoff_call(spot, k_call)

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
k_put = spot_ref * 0.95
k_call = spot_ref * 1.05
spot_ref, k_put, k_call

## Évolution du sous-jacent (SPY)
Historique des clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff du strangle long
Le payoff est la somme d'un put strike bas (OTM) et d'un call strike haut (OTM). La zone centrale perd la prime, tandis que les ailes captent une convexité plus prononcée qu'un straddle.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_strangle(S, k_put, k_call) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff strangle long")
ax.axvline(k_put, color="gray", linestyle=":", label=f"Strike put = {k_put:.2f}")
ax.axvline(k_call, color="gray", linestyle="--", label=f"Strike call = {k_call:.2f}")
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Strangle long sur SPY : payoff vs spot")
ax.legend()
plt.show()